In [35]:
# Setup for Ollama-based agent
import os

In [24]:
pip install -q langchain-ollama langchain-community langchain-core requests duckduckgo-search ddgs langchain langchain-hub

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement langchain-hub (from versions: none)
ERROR: No matching distribution found for langchain-hub
c:\Users\csinaywe\fastapi-workspace\langchain_models\venv\Lib\site-packages\IPython\utils\_process_win32.py:138: ResourceWarning: unclosed file <_io.BufferedWriter name=3>
  res = process_handler(cmd, _system_body)
c:\Users\csinaywe\fastapi-workspace\langchain_models\venv\Lib\site-packages\IPython\utils\_process_win32.py:138: ResourceWarning: unclosed file <_io.BufferedReader name=4>
  res = process_handler(cmd, _system_body)
c:\Users\csinaywe\fastapi-workspace\langchain_models\venv\Lib\site-packages\IPython\utils\_process_win32.py:138: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  res = process_handler(cmd, _system_body)


In [25]:
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
import requests

In [26]:
# Create a simple search tool to avoid network issues
@tool
def simple_search_tool(query: str) -> str:
    """Search for information. This provides basic knowledge for common queries."""
    # Simple knowledge base for common queries
    knowledge_base = {
        "capital of madhya pradesh": "Bhopal is the capital of Madhya Pradesh, India.",
        "madhya pradesh capital": "Bhopal is the capital of Madhya Pradesh, India.",
        "bhopal": "Bhopal is the capital city of Madhya Pradesh, India. It is known as the 'City of Lakes'.",
    }
    
    # Check for matches in our knowledge base
    query_lower = query.lower()
    for key, value in knowledge_base.items():
        if key in query_lower:
            return value
    
    # Default response for unknown queries
    return f"Search results for '{query}': This is a mock search result. In a real implementation, this would return web search results."

search_tool = simple_search_tool

In [27]:
@tool
def get_weather_data(city: str) -> str:
  """
  This function fetches the current weather data for a given city
  """
  url = f'https://api.weatherstack.com/current?access_key=8195461fb8916dfe75ddc9a8ce0acc22&query={city}'

  response = requests.get(url)

  return response.json()

In [28]:
llm = ChatOllama(
    model="llama3:latest", 
    temperature=0
)

In [29]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [30]:
# Step 2: Pull the ReAct prompt from LangChain Hub
prompt = hub.pull("hwchase17/react")  # pulls the standard ReAct agent prompt


c:\Users\csinaywe\fastapi-workspace\langchain_models\venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [31]:
# Step 3: Create the ReAct agent manually with the pulled prompt
agent = create_react_agent(
    llm=llm,
    tools=[search_tool, get_weather_data],
    prompt=prompt
)

In [32]:
# Step 4: Wrap it with AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool, get_weather_data],
    verbose=True
)

In [33]:
# Step 5: Invoke
response = agent_executor.invoke({"input": "Find the capital of Madhya Pradesh, then find it's current weather condition"})
print(response)



> Entering new AgentExecutor chain...
Thought: To find the capital of Madhya Pradesh and its current weather condition, I'll need to use a combination of tools.

Action: simple_search_tool
Action Input: "capital of Madhya Pradesh"Bhopal is the capital of Madhya Pradesh, India.Thought: To find the capital of Madhya Pradesh and its current weather condition, I'll need to use a combination of tools.

Action: simple_search_tool
Action Input: "capital of Madhya Pradesh"Bhopal is the capital of Madhya Pradesh, India.Thought: Now that I know the capital of Madhya Pradesh is Bhopal, I need to find its current weather condition.

Action: get_weather_data
Action Input: "Bhopal"Thought: Now that I know the capital of Madhya Pradesh is Bhopal, I need to find its current weather condition.

Action: get_weather_data
Action Input: "Bhopal"{'request': {'type': 'City', 'query': 'Bhopal, India', 'language': 'en', 'unit': 'm'}, 'location': {'name': 'Bhopal', 'country': 'India', 'region': 'Madhya Prades

In [34]:
response['output']

'The capital of Madhya Pradesh is Bhopal, and its current weather condition is Drizzle with a temperature of 26 degrees Celsius.'